In [1]:
from preprocess import *
from model import *
from plotr import *
import time

C:\Users\Ankit Kumar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Ankit Kumar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Preprocessing the preloaded style and content images 
## (from the images folder)

In [4]:
#preloaded content and style images
#To change go to images directory and replace the file name
content_1 = content_1
content_2 = content_2
style = style_image

In [6]:
#To preprocess images to fit the model requirements (VGG-19_ here)
content_image1 = preprocess_image(content_1)
content_image2 = preprocess_image(content_2)
style_image = preprocess_image(style_image)

In [10]:
#Initially setting the content image as generated image
generated_image = content_image1.clone().requires_grad_(True)

# An Instance of class that helps in clipping the features of hidden layers (declared in preprocess.py)

In [8]:
hooker = Hooker(vgg, style_layers + content_layers) #hooks feature to a dictionary and returns features
content_feats = hooker.extract_feats(content_image1) # feature extractor function
style_feats = hooker.extract_feats(content_image1)

In [9]:
#keys represent layer name
for name in vgg.named_modules():
    print(name)

('', Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): M

# Optimization loop for first image

In [6]:
content_losses = []
style_losses = []
total_losses = []
##########
#weights for light style fusion
##########
layers_weights = {
    0: 0.1,  # conv1_1
    5: 0.15, # conv2_1
    10: 0.2, # conv3_1
    19: 0.25, # conv4_1
    28: 0.3  # conv5_1
}
#########
"""for higher fusion of style in image use below"""
# layers_weights = {
#     0: 0.45,  # conv1_1
#     5: 0.65, # conv2_1
#     10: 0.85, # conv3_1
#     19: 0.45, # conv4_1
#     28: 0.55  # conv5_1
# }
# Training loop
optimizer = torch.optim.Adam([generated_image], lr=0.001)
alpha = 1e2  # Content loss weight preset for higher fusion
beta = 3e6  # Style loss weight
epochs = 10000
start_time = time.time()


for i in range(epochs):
    optimizer.zero_grad()
    optimizer.zero_grad()
    content_feats = hooker.extract_feats(content_image1)  
    style_feats = hooker.extract_feats(style_image)
    generated_feats = hooker.extract_feats(generated_image)
    J_content = compute_content_cost(content_feats, generated_feats)
    J_style = compute_style_cost(style_feats, generated_feats, layers_weights)
    J_total = alpha * J_content + beta * J_style
    J_total.backward()
    optimizer.step()
    generated_image.data.clamp_(0, 1)
    
    content_losses.append(J_content.item())
    style_losses.append(J_style.item())
    total_losses.append(J_total.item())
    
    if i % 100 == 0:
        print(f"Iteration {i}: Total Loss {J_total.item()}")
        img = postprocess_image(generated_image)
        img.save(f"output/output_1/Image_{i}.jpg")

end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")



Iteration 0: Total Loss 0.10525237768888474
Iteration 100: Total Loss 0.13217908143997192
Iteration 200: Total Loss 0.12047426402568817
Iteration 300: Total Loss 0.11706393957138062
Iteration 400: Total Loss 0.11527994275093079
Iteration 500: Total Loss 0.11411625146865845
Iteration 600: Total Loss 0.11323516070842743
Iteration 700: Total Loss 0.11240960657596588
Iteration 800: Total Loss 0.11160017549991608
Iteration 900: Total Loss 0.11072477698326111
Iteration 1000: Total Loss 0.10984527319669724
Iteration 1100: Total Loss 0.1089157983660698
Iteration 1200: Total Loss 0.10784554481506348
Iteration 1300: Total Loss 0.10669165849685669
Iteration 1400: Total Loss 0.10544496774673462
Iteration 1500: Total Loss 0.10428666323423386
Iteration 1600: Total Loss 0.10285582393407822
Iteration 1700: Total Loss 0.10135762393474579
Iteration 1800: Total Loss 0.09993645548820496
Iteration 1900: Total Loss 0.09849647432565689
Iteration 2000: Total Loss 0.09704551100730896
Iteration 2100: Total Loss

In [7]:
torch.save(style_feats, 'output/style_features.pt') 

In [8]:
plot(content_losses, style_losses, total_losses, 'output/loss_plot1.html') #saves the plot to destined path

In [9]:
plot_batched_loss(content_losses, style_losses, total_losses,"output/final_loss.html", batch_size=32) # save the batched plot (a better and reduced plot for better interpretability)

# Processing the second image and preforming Neural Style Tranfer

In [7]:

# style_image = preprocess_image(style_image)
generated_image2 = content_image2.clone().requires_grad_(True)

In [8]:
hooker = Hooker(vgg, style_layers + content_layers)
content_feats2 = hooker.extract_feats(content_image2)
style_feats = hooker.extract_feats(style_image)

# Optimization loop for second image

In [9]:
content_losses2 = []
style_losses2 = []
total_losses2 = []
##########
#weights for light style fusion
##########
layers_weights = {
    0: 0.1,  # conv1_1
    5: 0.15, # conv2_1
    10: 0.2, # conv3_1
    19: 0.25, # conv4_1
    28: 0.3  # conv5_1
}
#########
"""for higher fusion of style in image use below"""
# layers_weights = {
#     0: 0.45,  # conv1_1
#     5: 0.65, # conv2_1
#     10: 0.85, # conv3_1
#     19: 0.45, # conv4_1
#     28: 0.55  # conv5_1
# }
# Training loop
optimizer = torch.optim.Adam([generated_image2], lr=0.001)
alpha = 1e2  # Content loss weight preset for higher fusion
beta = 3e6  # Style loss weight
epochs = 10000

start_time = time.time()


for i in range(epochs):
    optimizer.zero_grad()
    optimizer.zero_grad()
    content_feats2 = hooker.extract_feats(content_image2)  
    style_feats = hooker.extract_feats(style_image)
    generated_feats2 = hooker.extract_feats(generated_image2)
    J_content = compute_content_cost(content_feats2, generated_feats2)
    J_style = compute_style_cost(style_feats, generated_feats2, layers_weights)
    J_total = alpha * J_content + beta * J_style
    J_total.backward()
    optimizer.step()
    generated_image2.data.clamp_(0, 1)
    
    content_losses2.append(J_content.item())
    style_losses2.append(J_style.item())
    total_losses2.append(J_total.item())
    
    if i % 100 == 0:
        print(f"Iteration {i}: Total Loss {J_total.item()}")
        img = postprocess_image(generated_image2)
        img.save(f"output/output_2/Image2_{i}.jpg")

end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

# Function to save the loss plot


Iteration 0: Total Loss 2.4520576000213623
Iteration 100: Total Loss 2.793419599533081
Iteration 200: Total Loss 2.6296563148498535
Iteration 300: Total Loss 2.558706760406494
Iteration 400: Total Loss 2.471503734588623
Iteration 500: Total Loss 2.3194544315338135
Iteration 600: Total Loss 2.12910532951355
Iteration 700: Total Loss 1.9499801397323608
Iteration 800: Total Loss 1.7834932804107666
Iteration 900: Total Loss 1.6249371767044067
Iteration 1000: Total Loss 1.4887752532958984
Iteration 1100: Total Loss 1.3817524909973145
Iteration 1200: Total Loss 1.2942020893096924
Iteration 1300: Total Loss 1.2204540967941284
Iteration 1400: Total Loss 1.15749990940094
Iteration 1500: Total Loss 1.1034027338027954
Iteration 1600: Total Loss 1.0553319454193115
Iteration 1700: Total Loss 1.0123648643493652
Iteration 1800: Total Loss 0.9732841849327087
Iteration 1900: Total Loss 0.9379203915596008
Iteration 2000: Total Loss 0.9063148498535156
Iteration 2100: Total Loss 0.8773205280303955
Iterati

# Change the output image size

In [8]:
interpolate_image("processed_image_path","save_image_path",("size"))

# Simple loss plot

In [10]:
filepath = "output/final_loss.html"  # Replace with the actual path
show_plot(filepath)

# Batched Loss plot 

In [11]:
filepath = "output/loss_plot1.html"  # Replace with the actual path
show_plot(filepath)